In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.nn import functional as F

from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd
import os
import pickle
import random
import itertools

In [2]:
a=np.load("/mnt/sd1/jhansi/interns/chaithra/MS/sal_project/Features/train/train_text_feats.npz", allow_pickle=True)['text_features']
b= np.load("/mnt/sd1/jhansi/interns/chaithra/MS/sal_project/Features/test/test__text_feats.npz", allow_pickle=True)['text_features']

t1=[np.squeeze(np.array(text), axis=1) for text in a]
t2 = [
    np.squeeze(np.array(text), axis=1)
    for text in b
    if len(np.array(text).shape) == 3
]

train_text_features=t1
test_text_features=t2
train_labels = np.load("/mnt/sd1/jhansi/interns/chaithra/MS/sal_project/Features/train/train_c_labels.npz", allow_pickle=True)['labels']
test_labels = np.load("/mnt/sd1/jhansi/interns/chaithra/MS/sal_project/Features/test/test_c_labels.npz", allow_pickle=True)['labels']

# Remove labels at indices 24 and 25
indices_to_remove = [24, 25]
test_labels = np.delete(test_labels, indices_to_remove, axis=0)

In [3]:
train_text_features_tensors = [torch.tensor(f) for f in train_text_features]
# Now pad the sequences to ensure uniform length
padded_train_text_features = pad_sequence(train_text_features_tensors, batch_first=True)

# Print the shape of the padded tensor
print(padded_train_text_features.shape)

test_text_features_tensors = [torch.tensor(f) for f in test_text_features]
padded_test_text_features = pad_sequence(test_text_features_tensors, batch_first=True)
print(padded_test_text_features.shape)


torch.Size([104, 69, 1024])
torch.Size([30, 63, 1024])


In [4]:
train_labels = np.array(train_labels, dtype=np.int64)
test_labels = np.array(test_labels, dtype=np.int64)


In [ ]:
import torch
from torch.utils.data import DataLoader

# Assuming train_text_features and test_text_features have been padded earlier
class textDataset(torch.utils.data.Dataset):
    def __init__(self, text_features, labels):
        self.text_features = text_features
        self.labels = labels

    def __len__(self):
        return len(self.text_features)

    def __getitem__(self, idx):
        # No need to wrap the features and labels again with torch.tensor
        return self.text_features[idx], torch.tensor(self.labels[idx], dtype=torch.long)

# Create train and test datasets
train_dataset = textDataset(padded_train_text_features, train_labels)
test_dataset = textDataset(padded_test_text_features, test_labels)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [7]:
from imblearn.over_sampling import SMOTE
import torch
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
labels_t = np.squeeze(train_labels)  # Converts to 1D if it's in shape (n_samples, 1)

# Flatten the audio features to 2D (n_samples, sequence_length * num_features)
flattened_train_text_features = padded_train_text_features.view(padded_train_text_features.size(0), -1)
print(flattened_train_text_features.shape)  # Should be [n_samples, 63 * 256]

# Apply SMOTE to balance the data
smote = SMOTE(random_state=42)
balanced_features, balanced_labels = smote.fit_resample(flattened_train_text_features.numpy(), labels_t)

# Convert the balanced features and labels to PyTorch tensors
balanced_features = torch.tensor(balanced_features).to(device)
balanced_labels = torch.tensor(balanced_labels).to(device)
# Check the current total size and calculate time steps
num_samples = balanced_features.size(0)
num_features = 1024  # Assuming this is fixed
time_steps = balanced_features.size(1) // num_features

# Reshape to 3D
balanced_features = balanced_features.view(num_samples, time_steps, num_features)
print(balanced_features.shape)  # Verify the shape



torch.Size([104, 70656])
torch.Size([148, 69, 1024])


In [8]:
import torch
from torch.utils.data import DataLoader

# Assuming train_text_features and test_text_features have been padded earlier
class textDataset(torch.utils.data.Dataset):
    def __init__(self, text_features, labels):
        self.text_features = text_features
        self.labels = labels

    def __len__(self):
        return len(self.text_features)

    def __getitem__(self, idx):
        # No need to wrap the features and labels again with torch.tensor
        return self.text_features[idx], torch.tensor(self.labels[idx], dtype=torch.long)

# Create train and test datasets
train_dataset = textDataset(balanced_features, balanced_labels)
test_dataset = textDataset(padded_test_text_features, test_labels)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
class TextBiLSTM(nn.Module):
    def __init__(self, config):
        super(TextBiLSTM, self).__init__()
        self.num_classes = config['num_classes']
        self.learning_rate = config['learning_rate']
        self.dropout = config['dropout']
        self.hidden_dims = config['hidden_dims']
        self.rnn_layers = config['rnn_layers']
        self.embedding_size = config['embedding_size']
        self.bidirectional = config['bidirectional']

        self.build_model()
        self.init_weight()
        
    def init_weight(net):
        for name, param in net.named_parameters():
            if 'ln' not in name:
                if 'bias' in name:
                    nn.init.constant_(param, 0.0)
                elif 'weight' in name:
                    nn.init.xavier_uniform_(param)

    def build_model(self):
        # attention layer
        self.attention_layer = nn.Sequential(
            nn.Linear(self.hidden_dims, self.hidden_dims),
            nn.ReLU(inplace=True)
        )
        # self.attention_weights = self.attention_weights.view(self.hidden_dims, 1)

        # 双层lstm
        self.lstm_net = nn.LSTM(self.embedding_size, self.hidden_dims,
                                num_layers=self.rnn_layers, dropout=self.dropout,
                                bidirectional=self.bidirectional)
                
        # FC层
        # self.fc_out = nn.Linear(self.hidden_dims, self.num_classes)
        self.fc_out = nn.Sequential(
            # nn.Dropout(self.dropout),
            nn.Linear(self.hidden_dims, self.hidden_dims),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.Linear(self.hidden_dims, self.num_classes),
            # nn.ReLU(),
            nn.Softmax(dim=1),
        )

        self.ln1 = nn.LayerNorm(self.embedding_size)
        self.ln2 = nn.LayerNorm(self.hidden_dims)

        
    def attention_net_with_w(self, lstm_out, lstm_hidden):
        '''
        :param lstm_out:    [batch_size, len_seq, n_hidden * 2]
        :param lstm_hidden: [batch_size, num_layers * num_directions, n_hidden]
        :return: [batch_size, n_hidden]
        '''
        lstm_tmp_out = torch.chunk(lstm_out, 2, -1)
        # h [batch_size, time_step, hidden_dims]
        h = lstm_tmp_out[0] + lstm_tmp_out[1]
        # h = lstm_out
        # [batch_size, num_layers * num_directions, n_hidden]
        lstm_hidden = torch.sum(lstm_hidden, dim=1)
        # [batch_size, 1, n_hidden]
        lstm_hidden = lstm_hidden.unsqueeze(1)
        # atten_w [batch_size, 1, hidden_dims]
        atten_w = self.attention_layer(lstm_hidden)
        # m [batch_size, time_step, hidden_dims]
        m = nn.Tanh()(h)
        # atten_context [batch_size, 1, time_step]
        atten_context = torch.bmm(atten_w, m.transpose(1, 2))
        # softmax_w [batch_size, 1, time_step]
        softmax_w = F.softmax(atten_context, dim=-1)
        # context [batch_size, 1, hidden_dims]
        context = torch.bmm(softmax_w, h)
        result = context.squeeze(1)
        return result

    def forward(self, x):
        # x : [len_seq, batch_size, embedding_dim]
        x = x.permute(1, 0, 2)
        # x = self.ln1(x)
        output, (final_hidden_state, _) = self.lstm_net(x)
        # output : [batch_size, len_seq, n_hidden * 2]
        output = output.permute(1, 0, 2)
        # final_hidden_state : [batch_size, num_layers * num_directions, n_hidden]
        final_hidden_state = final_hidden_state.permute(1, 0, 2)
        # final_hidden_state = torch.mean(final_hidden_state, dim=0, keepdim=True)
        # atten_out = self.attention_net(output, final_hidden_state)
        atten_out = self.attention_net_with_w(output, final_hidden_state)
        # atten_out = self.ln2(atten_out)
        return self.fc_out(atten_out)


In [22]:
config = {
    'num_classes': 2,
    'dropout': 0.5,
    'rnn_layers': 2,
    'embedding_size': 1024,
    'batch_size': 4,
    'epochs': 50,
    'learning_rate': 4e-5,
    'hidden_dims': 128,
    'bidirectional': True,
    'cuda': False,
}

In [23]:
model = TextBiLSTM(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to GPU

# Define optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
criterion = nn.CrossEntropyLoss()

In [24]:
# Training function

def train(epoch, train_loader,val_loader,best_accuracy):
    model.train()
    total_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        # Get sequence lengths (for packing)
        data = data.to(device)
        target = target.to(device)
        target = target.squeeze() 
        optimizer.zero_grad()
        output = model(data)
        
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    
    avg_loss = total_loss / len(train_loader)
    accuracy = correct / len(train_loader.dataset)
    
    model.eval()
    val_loss = 0
    val_correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data = data.to(device)
            target = target.to(device)
            target = target.squeeze()
            
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item()
            
            pred = output.argmax(dim=1, keepdim=True)
            val_correct += pred.eq(target.view_as(pred)).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = val_correct / len(val_loader.dataset)

    print(f"Train Epoch: {epoch}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), '/mnt/sd1/jhansi/interns/chaithra/MS/sal_project/Classification/Models/text_best_model_2.pth')  # Save the best model
    return best_accuracy

In [25]:
best_accuracy=0.0
for epoch in range(1, config['epochs'] + 1):
    best_accuracy=train(epoch, train_loader,test_loader,best_accuracy)
    print(f"Best model saved with accuracy: {best_accuracy:.4f}")


/tmp/ipykernel_69422/2750943036.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return self.text_features[idx], torch.tensor(self.labels[idx], dtype=torch.long)


Train Epoch: 1, Loss: 0.6945, Accuracy: 0.4797
Validation Loss: 0.6976, Validation Accuracy: 0.4000
Best model saved with accuracy: 0.4000
Train Epoch: 2, Loss: 0.6924, Accuracy: 0.4932
Validation Loss: 0.6993, Validation Accuracy: 0.4000
Best model saved with accuracy: 0.4000
Train Epoch: 3, Loss: 0.6934, Accuracy: 0.5068
Validation Loss: 0.6954, Validation Accuracy: 0.4000
Best model saved with accuracy: 0.4000
Train Epoch: 4, Loss: 0.6914, Accuracy: 0.5541
Validation Loss: 0.6967, Validation Accuracy: 0.4000
Best model saved with accuracy: 0.4000
Train Epoch: 5, Loss: 0.6937, Accuracy: 0.4932
Validation Loss: 0.7028, Validation Accuracy: 0.4000
Best model saved with accuracy: 0.4000
Train Epoch: 6, Loss: 0.6889, Accuracy: 0.5135
Validation Loss: 0.6996, Validation Accuracy: 0.4000
Best model saved with accuracy: 0.4000
Train Epoch: 7, Loss: 0.6903, Accuracy: 0.5541
Validation Loss: 0.6950, Validation Accuracy: 0.3333
Best model saved with accuracy: 0.4000
Train Epoch: 8, Loss: 0.684

In [26]:
from sklearn.metrics import f1_score, confusion_matrix,precision_score,recall_score

# Evaluation function
def evaluate(test_loader):
    model.eval()
    correct = 0
    total_loss = 0.0  
    total=0
    all_preds = []  # To store all predictions
    all_targets = []  # To store all true labels
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            target = target.squeeze()
            
            output = model(data)
            
            total += target.size(0)
            loss = criterion(output, target)
            total_loss += loss.item()
            
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            all_preds.extend(pred.cpu().numpy().flatten())
            all_targets.extend(target.cpu().numpy().flatten())
            
        accuracy = correct / total
        average_loss = total_loss / len(test_loader)
        
        results_df = pd.DataFrame({
        'Predicted': all_preds,
        'Target': all_targets
        })
    
        #print("\nPredicted vs Target:")
        #print(results_df)
        
        f1 = f1_score(all_targets, all_preds, average='weighted')
        cm = confusion_matrix(all_targets, all_preds)
        precision = precision_score(all_targets, all_preds, average='weighted')
        recall = recall_score(all_targets, all_preds, average='weighted')
        
    print(f"Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
    
    print("Confusion Matrix:")
    print(cm)
    return accuracy,results_df

In [27]:
########## data aug ########
model.load_state_dict(torch.load('/mnt/sd1/jhansi/interns/chaithra/MS/sal_project/Classification/Models/text_best_model_2.pth', weights_only=True))
model.eval()  # Set to evaluation mode

# Evaluate on the test set
accuracy = evaluate(test_loader)

Loss: 0.5448, Accuracy: 0.7667
F1 Score: 0.7647, Precision: 0.7646, Recall: 0.7667
Confusion Matrix:
[[15  3]
 [ 4  8]]
